<a href="https://colab.research.google.com/github/cheng1103/Eco-Fin_News-Crawler-Test/blob/main/Cnyes_category_fast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><span style="font-size:20px"><strong>鉅亨網新聞主題爬蟲（精確版，用於Colab平台）</strong></span></h1>

<p>可自訂主題新聞與日期區間爬取新聞內文，並可設定是否爬取頭條以外的新聞<br />
<span style="color:null"><strong>輸出資料的欄位依序為：</strong></span> <strong>張貼時間、新聞編號(網址)、版別、標題、頭條標記、內文、股號(如果有)</strong> <strong>。資料格式為csv檔，並以UTF8編碼，直接存入Google雲端硬碟</strong></p>

<p>支援的分類新聞目錄：</p>

<p>1.台股 V<br />
&nbsp; a.台灣政經 V<br />
&nbsp; b.台股盤勢 V<br />
&nbsp; c.台股盤勢 V<br />
&nbsp; d.台股盤勢 V</p>

<p>2.國際股 V<br />
&nbsp; a.美股 V<br />
&nbsp; b.美股雷達 V<br />
&nbsp; c.歐亞股 V<br />
&nbsp; d.國際政經 V</p>

<p>3.陸港股 V</p>

<p>4.外匯 V</p>

<p>5.期貨 V</p>

<p>6.房產 V</p>

<p>7.基金 V</p>

<p>8.區塊鏈 V</p>


# 新聞類別與查詢區間設定


In [75]:
#設定區域 publishAt 	newsId 	categoryName 	title 	isCategoryHeadline 	content 	stock
#==============================================
#頭條設定 
# 1表示限定頭條； 0表示不限 
Headline = 1

#輸入類別
c = '台股'

#查詢區間
s_time = "2021-09-01 00:00:00" #開始日期 
e_time = "2022-01-31 00:00:00" #結束日期

#檔名設定
file_name = 'original_data'

#==============================================

data_URL = []

kind = {
  '台股':'tw_stock',
  '台灣政經':'tw_macro',
  '台股盤勢':'tw_quo',
  '專家觀點':'stock_report',
  '台股新聞':'tw_stock_news',

  '國際股':'wd_stock',
  '美股':'us_stock',
  '美股雷達':'us_stock_live',
  '歐亞股':'eu_asia_stock',
  '國際政經':'wd_macro',

  '陸港股':'cn_stock',
  '外匯':'forex',
  '期貨':'future',
  '房產':'cnyeshouse',
  '基金':'fund',
  '區塊鏈':'bc'

}

c = kind.get(c)


#頭條篩選
if Headline == 1 :
  CH = '&isCategoryHeadline=1'
else :
  CH = ''

# 安裝並載入套件

In [48]:
!pip install fake_useragent
from fake_useragent import UserAgent
import requests
from bs4 import BeautifulSoup
from lxml import etree

import random
import time
import datetime
import json
import pandas as pd
import re
#%reset -f

# 自定義程式

In [82]:
def url_list(s_time,e_time):
  
  #==============================================
  #時間區間設定
  s_time = int(time.mktime(time.strptime(s_time, "%Y-%m-%d %H:%M:%S")))-28800 # 轉成時間戳
  e_time = int(time.mktime(time.strptime(e_time, "%Y-%m-%d %H:%M:%S")))-28800

  #==============================================
  p = 1
  l_page = 9
  data = pd.DataFrame()

  while p <= l_page :

    headers = UserAgent().random #隨機請求表頭
    
    url = 'https://api.cnyes.com/media/api/v1/newslist/category/'+c+'?startAt='+str(s_time)+'&endAt='+str(e_time)+CH+"&limit=30&page="+str(p)
    req = requests.get(url, headers={"User-Agent":headers})
    req = req.json()

    #最末頁之路徑
    if p == 1:
      l_page = req["items"]["last_page"] 

    req_data = req["items"]["data"]
    temp_data = pd.DataFrame.from_dict(req_data)
    
    #擷取資料
    publishAt = temp_data['publishAt']
    newsId_url = temp_data['newsId']
    categoryName = temp_data['categoryName']
    title = temp_data['title']
    Headline = temp_data['isCategoryHeadline']
    stock = temp_data['stock']
    #stock =  ",".join(stock)
    content = temp_data['content']

    temp_data = pd.concat([publishAt, newsId_url, categoryName, title, Headline ,content, stock], 1)
    temp_data = reFormat(temp_data)







    data = pd.concat([data, temp_data])
        
    #休眠
    t = random.uniform(0.005, 0.2) #隨機數
    time.sleep(t) #暫停

    p = p+1

    if p%20 == 0:
      print(p/l_page*100)
      data.drop_duplicates(subset='newsId', keep='first', inplace=True) #移除重複資料
      time.sleep(0.5)


  data.drop_duplicates(subset='newsId', keep='first', inplace=True) #移除重複資料
  return(data)



In [83]:
def reFormat (temp_data):
  for r in range(len(temp_data)):
      c = temp_data['content'][r]
      c = etree.HTML(text=c)
      c = c.xpath('string(.)')
      pattern = re.compile(r'<[^>]+>',re.S)
      c = pattern.sub('', c)
      c = c.replace('\n', '').replace('&nbsp', '')
      temp_data['content'][r] = c

      struct_time = time.localtime(temp_data['publishAt'][r]) # 轉成時間元組
      temp_data['publishAt'][r] = time.strftime("%Y/%m/%d %H:%M", struct_time) # 轉成字串
  return(temp_data)

# 執行程式

In [84]:
data = url_list(s_time,e_time)
data = data.sort_values(by='publishAt', ascending=True)
data = data.reset_index(drop=True) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the c

19.607843137254903
39.21568627450981
58.82352941176471
78.43137254901961
98.0392156862745


# 檢視資料

In [85]:
data

,publishAt,newsId,categoryName,title,isCategoryHeadline,content,stock
0,2021/11/30 22:00,4778535,台股新聞,〈台股盤前要聞〉台積電再發債167億元、弘塑扮異質整合要角 今日必看財經新聞,1,關注台股盤前要聞重點，因應新擴建廠房設備，台積電再發行 167 億元無擔保公司債；台積電異質...,"[2317, 2330, 2881, 2888, 3090, 3131, 3680, 5820]"
1,2021/12/01 00:00,4778582,台股新聞,群益權證小教室：技術面－KD高檔鈍化,1,歡迎來到群益權證小教室！「權證」是很適合小資族的投資工具，有機會透過小額資金創造更多收入，今...,[6182]
2,2021/12/01 00:00,4778242,專家觀點,操盤手看台股：元宇宙股果然舊不如新 下一步怎麼選？,1,面對來勢洶洶的 Omicron 新變種病毒株，美國總統拜登表示不考慮旅行禁令或封城，此舉讓市...,"[2497, 2498, 2618, 3701, 5371]"
3,2021/12/01 00:30,4778591,台股新聞,Omicron變種病毒引爆獲利新契機,1,在莫德納執行長 Stephane Bancel 表示現有疫苗恐不足以對抗 Omicron 病...,"[3701, 2497, 3624, 8046, 3037, 2401, 3006, 237..."
4,2021/12/01 00:38,4778601,台股盤勢,〈台股盤前〉聯準會考慮加速Taper美股大跌 上檔有壓估維持區間震盪,1,台股昨 (11 月 30 日) 開高，但受 MSCI 調整指數收盤生效影響，尾盤爆量、漲幅收...,"[2330, 3491, 4916, 6271]"
...,...,...,...,...,...,...,...
3032,2022/01/31 08:00,4807605,台股新聞,〈企業衝生育率〉晶技育兒津貼方案 前三胎補助每月3000元起,1,物價、房價不斷上漲，台灣生育率逐年降低，甚至可能形成國安危機，政府、企業無不費盡心思祭出「催...,[3042]
3033,2022/01/31 09:00,4808165,台股新聞,〈企業衝生育率〉鴻海「0到6歲公司養」一胎補助130.5萬元 已發逾3億元,1,鴻海 (2317-TW) 積極推動生育政策，自 2020 年起推動「0 到 6 歲公司養」政...,[2317]
3034,2022/01/31 10:00,4802091,台股新聞,〈企業衝生育率〉中華電獨步全台減少員工工時 台灣大津貼最高12萬元,1,電信商為打造友善職場環境，今年全面提升育兒福利政策，中華電 (2412-TW) 獨步全台，有...,"[2412, 3045, 4904]"
3035,2022/01/31 10:30,4809088,台股新聞,〈企業衝生育率〉多家券商補助生育津貼 最高達10萬元,1,據統計，國內去年全年出生人數 15 萬 3820 人，創史上新低，人口負成長壓力持續擴大，因...,[]


# 連結Google Drive並匯出資料

In [ ]:

from google.colab import drive
drive.mount('/content/drive')


#輸出檔案
path = '/content/drive/MyDrive/'+file_name+'.csv'
data.to_csv(path)

Mounted at /content/drive
